In [ ]:
import datasets

dataset = datasets.load_dataset('FinGPT/fingpt-finred')
# save to local disk space (recommended)
dataset.save_to_disk('data/fingpt-finred')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/745 [00:00<?, ?B/s]

(…)-00000-of-00001-4af4d9c6fa204dbc.parquet:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

(…)-00000-of-00001-34e4fcf97bb5994c.parquet:   0%|          | 0.00/354k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27558 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5112 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27558 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5112 [00:00<?, ? examples/s]

Single Hop

In [ ]:
dataset['test']

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 5112
})

In [ ]:
forecaster = datasets.load_dataset('FinGPT/fingpt-forecaster-dow30-202305-202405')

README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

(…)-00000-of-00001-7c4c80aa07272d4c.parquet:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

(…)-00000-of-00001-28531804b005ddc6.parquet:   0%|          | 0.00/925k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1230 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
forecaster['test']

Dataset({
    features: ['prompt', 'answer', 'period', 'label', 'symbol'],
    num_rows: 300
})

In [ ]:
from datasets import load_dataset

qa = load_dataset("FinGPT/fingpt-fiqa_qa")

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

(…)-00000-of-00001-ab79bf9300210e98.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [ ]:
qa['train']

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 17110
})

In [ ]:
import random
from datasets import Dataset, concatenate_datasets

qa = qa['train']
forecaster = forecaster['test']
dataset = dataset['test']


n_qa_samples = 300
n_forecaster_samples = 300
n_finred_samples = 300

qa_indices = random.sample(range(len(qa)), n_qa_samples)
forecaster_indices = random.sample(range(len(forecaster)), n_forecaster_samples)
finred_indices = random.sample(range(len(dataset)), n_finred_samples)


In [ ]:
qa_sampled = qa.select(qa_indices)
forecaster_sampled = forecaster.select(forecaster_indices)
finred_sampled = dataset.select(finred_indices)

In [ ]:
# align with other 2 dataset format
forecaster_mapped = forecaster_sampled.map(
    lambda example: {
        'input': example['prompt'],
        'output': example['answer'],
        'instruction': f"Forecast for {example['symbol']} in {example['period']}"
    },
    remove_columns=['prompt', 'answer', 'period', 'label', 'symbol']
)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
forecaster_mapped

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 300
})

In [ ]:
combined_dataset = concatenate_datasets([qa_sampled, finred_sampled, forecaster_mapped])


In [ ]:

combined_dataset[0]

{'input': 'Is freelance income earned by a U.S. citizen while living abroad subject to state income tax?',
 'output': 'No state taxes, but Italy also has a favorable treaty with the US Federal Government. Look into to lowering your federal taxes to 5% ;) its a thick read, http://www.irs.gov/businesses/international/article/0,,id=169601,00.html and also try to determine if the Foreign Earned Income Exclusion applies to you, reducing your Federal tax to ZERO on the first $95,100 earned abroad. http://www.irs.gov/businesses/small/international/article/0,,id=97130,00.html but then you may be subject to a 20%+ italy tax. so maybe you should just try for the tax treaty',
 'instruction': 'Utilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.'}

In [ ]:
combined_dataset.to_json("combined_financial_dataset.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3074005

In [ ]:
len(combined_dataset)

900

# Multi hop

In [5]:
import openai
import json





with open('/content/clean_router_testset_60.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]


finqa_questions = []
finred_questions = []
forecaster_questions = []

for entry in data:
    if entry['module'] == 'FinQA':
        finqa_questions.append(entry)
    elif entry['module'] == 'FinRED':
        finred_questions.append(entry)
    elif entry['module'] == 'Forecaster':
        forecaster_questions.append(entry)


In [4]:
data

[{'input': 'Who is the headquarters location of Goldman Sachs?',
  'output': 'headquarters_location: Goldman Sachs, New York',
  'instruction': 'Extract structured relations about companies or financial entities mentioned in the input.',
  'module': 'FinRED'},
 {'input': 'Who is the founded by of Netflix?',
  'output': 'founded_by: Netflix, Reed Hastings',
  'instruction': 'Extract structured relations about companies or financial entities mentioned in the input.',
  'module': 'FinRED'},
 {'input': 'Who is the stock exchange of Apple?',
  'output': 'stock_exchange: Apple, NASDAQ',
  'instruction': 'Extract structured relations about companies or financial entities mentioned in the input.',
  'module': 'FinRED'},
 {'input': 'Who is the chief executive officer of Google?',
  'output': 'chief_executive_officer: Google, Sundar Pichai',
  'instruction': 'Extract structured relations about companies or financial entities mentioned in the input.',
  'module': 'FinRED'},
 {'input': 'Will the D

In [3]:
len(finqa_questions), len(finred_questions), len(forecaster_questions)

(20, 20, 20)

In [7]:

from openai import OpenAI
api_key = ''

multi_hop_questions = []

import os
import openai

# Set the environment variable directly in Colab
os.environ["LITELLM_API_KEY"] = "sk-sfbqEWDJdZJYd_bklSlcrw"  # Replace with your actual API key

# Initialize the client
client = openai.OpenAI(
  api_key=os.environ.get("LITELLM_API_KEY"),
  base_url="https://cmu.litellm.ai",
)

def generate_multihop_question(q1, q2):
    prompt = f"Create a multi-hop financial question that combines the following two questions into a complex, multi-step question:\n\n1. {q1}\n2. {q2}\n\nReturn the combined multi-hop question."

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )

    return response.choices[0].message.content


In [26]:
# Example: Combine questions from different modules (FinQA + Forecaster)
for q1 in finqa_questions[:5]:
    for q2 in forecaster_questions[:5]:
        multi_hop_question = generate_multihop_question(q1['input'], q2['input'])
        multi_hop_questions.append({
            'original_questions': [
                {'input': q1},
                {'input': q2}
            ],
            'multi_hop_question': multi_hop_question
        })

In [27]:
multi_hop_questions


[{'original_questions': [{'input': {'input': 'How can I improve my credit score quickly?',
     'output': 'It depends on your goals, but consider strategies like diversification, tax planning, and emergency savings.',
     'instruction': 'Utilize your financial knowledge, give your answer or opinion to the input question or subject. Answer format is not limited.',
     'module': 'FinQA'}},
   {'input': {'input': 'Will the Dow Jones rise or fall next week?',
     'output': '[Positive Developments]: Strong earnings, favorable macro trends. [Potential Concerns]: Technical resistance, volatility.\n\n[Prediction & Analysis]: Prediction: Up 2–4%. Analysis: Market sentiment and fundamentals suggest modest gains.',
     'instruction': 'Based on your knowledge, forecast the stock or crypto price movement and provide a brief analysis.',
     'module': 'Forecaster'}}],
  'multi_hop_question': 'How can I quickly improve my credit score in order to qualify for a better interest rate on a loan, and 

In [28]:
for q1 in finred_questions[:5]:
    for q2 in forecaster_questions[:5]:
        multi_hop_question = generate_multihop_question(q1['input'], q2['input'])
        multi_hop_questions.append({
            'original_questions': [
                {'input': q1},
                {'input': q2}
            ],
            'multi_hop_question': multi_hop_question
        })

In [ ]:
for q1 in finqa_questions[:5]:
    for q2 in finred_questions[:5]:
        multi_hop_question = generate_multihop_question(q1['input'], q2['input'])
        multi_hop_questions.append({
            'original_questions': [
                {'input': q1},
                {'input': q2}
            ],
            'multi_hop_question': multi_hop_question
        })

In [1]:
import json
with open('multi_hop_questions.json', 'r') as f:
    multi_hop_questions = json.load(f)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizerFast, LlamaForCausalLM
from peft import PeftModel
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

def load_forecaster_model():
    print("Loading FinGPT Forecaster model...")
    tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
    base_model = AutoModelForCausalLM.from_pretrained(
        'meta-llama/Llama-2-7b-chat-hf',
        trust_remote_code=True,
        device_map="auto",
        quantization_config=bnb_config
    )
    model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
    return model, tokenizer



def load_finred_model():
    print("Loading FinGPT Relation Extraction model...")
    base_model = 'llama2'
    peft_model = 'FinGPT/fingpt-mt_llama2-7b_lora'
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    base_model = AutoModelForCausalLM.from_pretrained(base_model)
    model = PeftModel.from_pretrained(base_model, peft_model, quantization_config=bnb_config)
    return model, tokenizer

def load_finqa_model():
    print("Loading FinGPT QA model...")
    base_model = "meta-llama/Meta-Llama-3.1-8B"
    peft_model = "FinGPT/fingpt-mt_llama3-8b_lora"
    tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map="cuda:0")
    model = PeftModel.from_pretrained(model, peft_model, quantization_config=bnb_config)
    return model, tokenizer



In [3]:
# Function to route multi-hop questions
def route_question(question):
    routing_prompt = f"""
    Analyze the following multi-hop financial question and determine which two specialized financial models it should be routed to:

    Question: {question}

    Categorize the question into TWO of these categories:
    1. Financial QA (FinQA) - Basic financial knowledge questions
    2. Financial Relation Extraction (FinRed) - Questions about relationships between financial entities
    3. Stock Movement Prediction (FinForecaster) - Questions about stock market predictions

    Provide your routing decision as a JSON object with these fields:
    - "first_model": First model to route to (FinQA, FinRed, or FinForecaster)
    - "first_subquestion": The part of the question that should be answered by the first model
    - "second_model": Second model to route to (FinQA, FinRed, or FinForecaster)
    - "second_subquestion": The part of the question that should be answered by the second model
    - "routing_explanation": Brief explanation for your routing decision

    Format your response as valid JSON only with no additional text.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Specify the actual model identifier for GPT-4o-mini
            messages=[
                {"role": "system", "content": "You are a specialized financial question router."},
                {"role": "user", "content": routing_prompt}
            ],
            response_format={"type": "json_object"}
        )

        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Error in routing: {e}")
        return None


def process_with_models(routing_result, models, tokenizers):
    results = {
        "routing_decision": routing_result,
        "model_responses": {}
    }

    # Process with first model
    first_model_name = routing_result["first_model"]
    if first_model_name in models:
        print(f"Processing with {first_model_name}...")
        model, tokenizer = models[first_model_name], tokenizers[first_model_name]

        if first_model_name == "FinQA":
            prompt = f"Instruction: Utilize your financial knowledge, give your answer or opinion to the input question or subject. Answer format is not limited.\nInput: {routing_result['first_subquestion']}"
        elif first_model_name == "FinRed":
            prompt = f"Extract structured relations about companies or financial entities mentioned in the input: {routing_result['first_subquestion']}"
        else:
            prompt = f"Based on your knowledge, forecast the stock or crypto price movement and provide a brief analysis. Input: {routing_result['first_subquestion']}"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=512,
                temperature=0.7
            )
        first_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results["model_responses"][first_model_name] = {
            "question": routing_result["first_subquestion"],
            "response": first_response
        }

    # Process with second model
    second_model_name = routing_result["second_model"]
    if second_model_name in models:
        print(f"Processing with {second_model_name}...")
        model, tokenizer = models[second_model_name], tokenizers[second_model_name]

        first_response = results["model_responses"].get(first_model_name, {}).get("response", "")
        if second_model_name == "FinQA":
            prompt = f"Instruction: Utilize your financial knowledge, give your answer or opinion to the input question or subject. Answer format is not limited.\nPrevious context: {first_response}\nInput: {routing_result['second_subquestion']}"
        elif second_model_name == "FinRed":
            prompt = f"Previous context: {first_response}\nExtract financial relations from the following: {routing_result['second_subquestion']}"
        else:
            prompt = f"Instruction: Based on your knowledge, forecast the stock or crypto price movement and provide a brief analysis.\nPrevious context: {first_response}\nInput: {routing_result['second_subquestion']}"

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=512,
                temperature=0.7
            )
        second_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results["model_responses"][second_model_name] = {
            "question": routing_result["second_subquestion"],
            "response": second_response
        }

    # Evaluate the multi-hop answer with the router
    combined_responses = f"First response ({first_model_name}): {results['model_responses'].get(first_model_name, {}).get('response', 'No response')}\n\nSecond response ({second_model_name}): {results['model_responses'].get(second_model_name, {}).get('response', 'No response')}"

    eval_prompt = f"""
    Evaluate the quality of the following multi-hop answer to the original question.

    Original question: {routing_result.get('original_question', 'Unknown')}

    {combined_responses}

    Provide a brief evaluation of:
    1. Accuracy of responses
    2. Completeness of the answer
    3. Integration between the two model responses
    4. Overall quality on a scale of 1-10

    Format your evaluation as JSON.
    """

    try:
        eval_response = router_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a financial answer evaluator."},
                {"role": "user", "content": eval_prompt}
            ],
            response_format={"type": "json_object"}
        )

        results["evaluation"] = json.loads(eval_response.choices[0].message.content)
    except Exception as e:
        print(f"Error in evaluation: {e}")
        results["evaluation"] = {"error": str(e)}

    return results



                

In [11]:
model_loaders = {
        "FinQA": load_finqa_model,
        "FinRed": load_finred_model,
        "FinForecaster": load_forecaster_model
    }
def process_multi_hop_questions(questions_dataset):
    models = {}
    tokenizers = {}

    results = []

    for idx, item in enumerate(questions_dataset):
        multi_hop_question = item['multi_hop_question']
        print(f"\nProcessing question {idx+1}: {multi_hop_question}")

        # Route the question
        routing_result = route_question(multi_hop_question)
        

        # Add original question to routing result
        routing_result["original_question"] = multi_hop_question

#         # Load models on demand
#         for model_name in [routing_result["first_model"], routing_result["second_model"]]:
#             if model_name not in models and model_name in model_loaders:
#                 model, tokenizer = model_loaders[model_name]()
#                 models[model_name] = model
#                 tokenizers[model_name] = tokenizer

#         # Process with models
#         question_results = process_with_models(routing_result, models, tokenizers)

#         # Compare to original subquestions if available
#         if 'original_questions' in item:
#             question_results["original_subquestions"] = item['original_questions']

        results.append(routing_result)

#         # Save results periodically
#         if (idx + 1) % 5 == 0 or idx == len(questions_dataset) - 1:
#             with open(f"multi_hop_results_{idx+1}.json", "w") as f:
#                 json.dump(results, f, indent=2)

    return results

In [4]:
from huggingface_hub import login

login()  # It will prompt you to paste your Hugging Face access token


Token has not been saved to git credential helper.


In [6]:
import os
import openai

# Set the environment variable directly in Colab
os.environ["LITELLM_API_KEY"] = "sk-sfbqEWDJdZJYd_bklSlcrw"  # Replace with your actual API key

# Initialize the client
client = openai.OpenAI(
  api_key=os.environ.get("LITELLM_API_KEY"),
  base_url="https://cmu.litellm.ai",
)

In [12]:
route_results = process_multi_hop_questions(multi_hop_questions[:10])


Processing question 1: How can I quickly improve my credit score in order to qualify for a better interest rate on a loan, and how might the anticipated movements of the Dow Jones next week influence my decision to apply for that loan?

Processing question 2: What immediate steps can I take to quickly improve my credit score, and how might these actions influence my ability to invest in Microsoft stock, particularly considering the potential for its price to fluctuate next week?

Processing question 3: What steps can I take to quickly improve my credit score, and how might these actions influence my ability to invest in Meta shares, considering the potential for their price to rise next week?

Processing question 4: What are some effective strategies to quickly improve my credit score, and given those improvements, how might a better credit profile influence my investment decisions, particularly regarding whether I should invest in Bitcoin if it is expected to break past its resistanc

In [17]:
torch.cuda.empty_cache()

In [13]:
route_results2 = process_multi_hop_questions(multi_hop_questions[10:])


Processing question 1: Given the current volatility in the market, how should I strategically allocate my 401(k) investments to minimize risk and maximize potential returns, while also considering predictions regarding whether the Dow Jones is likely to rise or fall next week?

Processing question 2: In the context of navigating a volatile market, how should I allocate my 401(k) investments to optimize stability and growth, considering the potential short-term fluctuations in the stock market, including the specific forecast for Microsoft stock, which is expected to either rise or fall next week?

Processing question 3: In a volatile market, what strategies should I employ to allocate my 401(k) effectively, particularly if I am considering investing a portion of my contributions in Meta shares, and based on current market trends and analyst predictions, what is the likelihood that Meta shares will rise next week, influencing my allocation decision?

Processing question 4: Given the cu


Processing question 29: What impact might the performance of Apple’s stock on its respective exchange have on investor sentiment towards Bitcoin, and how could this sentiment influence whether Bitcoin breaks past its resistance level next week?

Processing question 30: What is the stock exchange where Apple is listed, and based on current market trends, is Salesforce expected to drop in value next week?

Processing question 31: What impact might the leadership of Google's CEO have on investor sentiment towards major technology stocks, and how could this sentiment influence the Dow Jones Industrial Average's performance in the upcoming week?

Processing question 32: What impact might the leadership of Google's CEO have on the tech industry's overall performance, and how could this influence investor sentiment regarding Microsoft's stock movement next week?

Processing question 33: What impact does the leadership of Google, specifically the chief executive officer, have on investor sent

In [14]:
route_results

[{'first_model': 'FinQA',
  'first_subquestion': 'How can I quickly improve my credit score in order to qualify for a better interest rate on a loan',
  'second_model': 'FinForecaster',
  'second_subquestion': 'how might the anticipated movements of the Dow Jones next week influence my decision to apply for that loan',
  'routing_explanation': 'The first part of the question is about improving credit score, which falls under basic financial knowledge, while the second part relates to predicting stock market movements and how they affect loan decisions.',
  'original_question': 'How can I quickly improve my credit score in order to qualify for a better interest rate on a loan, and how might the anticipated movements of the Dow Jones next week influence my decision to apply for that loan?'},
 {'first_model': 'FinQA',
  'first_subquestion': 'What immediate steps can I take to quickly improve my credit score?',
  'second_model': 'FinForecaster',
  'second_subquestion': 'How might these act

In [33]:
import json
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import difflib
from sentence_transformers import SentenceTransformer
import torch
from scipy.spatial.distance import cosine

def evaluate_router_accuracy(multi_hop_questions, route_results):
    """
    Evaluate the accuracy of the router by comparing its decisions to ground truth.
    
    Args:
        multi_hop_questions: List of questions with original subquestions
        route_results: List of routing decisions from the router
    
    Returns:
        Dictionary with various accuracy metrics
    """
    # Load sentence transformer model for semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    results = {
        "model_classification_accuracy": 0,
        "semantic_similarity_scores": [],
        "exact_match_rate": 0,
        "confusion_matrix": {
            "FinQA": {"FinQA": 0, "FinRed": 0, "FinForecaster": 0},
            "FinRed": {"FinQA": 0, "FinRed": 0, "FinForecaster": 0},
            "FinForecaster": {"FinQA": 0, "FinRed": 0, "FinForecaster": 0}
        },
        "detailed_comparisons": []
    }
    
    correct_model_classifications = 0
    exact_matches = 0
    total_evaluations = 0
    
    for i, (question, route_result) in enumerate(zip(multi_hop_questions, route_results)):
        if 'original_questions' not in question:
            continue
            
        # Extract ground truth
        ground_truth_models = []
        ground_truth_questions = []
        
        for subq in question['original_questions']:
            module = subq['input'].get('module')
            if module:
                # Convert module names if needed
                if module == "Forecaster":
                    module = "FinForecaster"
                ground_truth_models.append(module)
                ground_truth_questions.append(subq['input'].get('input', ''))
        
        # Extract router predictions
        ground_truth_models = [model.replace('FinRED', 'FinRed') for model in ground_truth_models]
        predicted_models = [route_result.get('first_model'), route_result.get('second_model')]
        predicted_questions = [route_result.get('first_subquestion'), route_result.get('second_subquestion')]
        
        # Compute model classification accuracy
        # First, try direct matching
        model_correct = 0
        for gt_model in ground_truth_models:
            if gt_model in predicted_models:
                model_correct += 1
                # Update confusion matrix (true positive)
                results["confusion_matrix"][gt_model][gt_model] += 1
            else:
                # Update confusion matrix (false negative)
                for pred_model in predicted_models:
                    if pred_model != gt_model:
                        results["confusion_matrix"][gt_model][pred_model] += 1
        
        model_accuracy = model_correct / len(ground_truth_models) if ground_truth_models else 0
        correct_model_classifications += model_correct
        
        # Compute semantic similarity between questions
        # First, get embeddings
        gt_embeddings = model.encode(ground_truth_questions)
        pred_embeddings = model.encode(predicted_questions)
        
        # Calculate similarities
        max_similarities = []
        question_matches = []
        
        for j, gt_emb in enumerate(gt_embeddings):
            similarities = [1 - cosine(gt_emb, pred_emb) for pred_emb in pred_embeddings]
            max_sim = max(similarities)
            max_idx = similarities.index(max_sim)
            max_similarities.append(max_sim)
            
            # Check for exact match
            if ground_truth_questions[j].lower() == predicted_questions[max_idx].lower():
                exact_matches += 1
                question_match = "EXACT"
            else:
                question_match = "SIMILAR" if max_sim > 0.8 else "DIFFERENT"
            
            # Store comparison details
            question_matches.append({
                "ground_truth": ground_truth_questions[j],
                "predicted": predicted_questions[max_idx],
                "similarity": max_sim,
                "match_type": question_match
            })
        
        # Calculate average similarity for this question
        avg_similarity = sum(max_similarities) / len(max_similarities) if max_similarities else 0
        results["semantic_similarity_scores"].append(avg_similarity)
        
        # Add detailed comparison for this question
        results["detailed_comparisons"].append({
            "multi_hop_question": question.get('multi_hop_question', ''),
            "ground_truth_models": ground_truth_models,
            "predicted_models": predicted_models,
            "model_accuracy": model_accuracy,
            "avg_semantic_similarity": avg_similarity,
            "question_comparisons": question_matches
        })
        
        total_evaluations += len(ground_truth_models)
    
    # Calculate overall metrics
    if total_evaluations > 0:
        results["model_classification_accuracy"] = correct_model_classifications / total_evaluations
        results["exact_match_rate"] = exact_matches / total_evaluations
    
    results["avg_semantic_similarity"] = sum(results["semantic_similarity_scores"]) / len(results["semantic_similarity_scores"]) if results["semantic_similarity_scores"] else 0
    
    return results

def generate_accuracy_report(evaluation_results):
    """Generate a human-readable report of router accuracy"""
    report = []
    report.append("# Financial Question Router Accuracy Report\n")
    
    # Overall metrics
    report.append("## Overall Metrics")
    report.append(f"- Model Classification Accuracy: {evaluation_results['model_classification_accuracy']:.2%}")
    report.append(f"- Average Semantic Similarity: {evaluation_results['avg_semantic_similarity']:.2%}")
    report.append(f"- Exact Question Match Rate: {evaluation_results['exact_match_rate']:.2%}\n")
    
    # Confusion Matrix
    report.append("## Model Type Confusion Matrix")
    report.append("```")
    report.append("Predicted →  | FinQA | FinRed | FinForecaster")
    report.append("-------------|-------|--------|-------------")
    for actual_model in ["FinQA", "FinRed", "FinForecaster"]:
        matrix_row = evaluation_results["confusion_matrix"][actual_model]
        report.append(f"{actual_model} | {matrix_row['FinQA']} | {matrix_row['FinRed']} | {matrix_row['FinForecaster']}")
    report.append("```\n")
    
    # Detailed examples
    report.append("## Detailed Examples")
    for i, comparison in enumerate(evaluation_results["detailed_comparisons"][:5]):  # Show first 5 examples
        report.append(f"### Example {i+1}")
        report.append(f"**Multi-hop Question:** {comparison['multi_hop_question']}")
        report.append(f"**Ground Truth Models:** {', '.join(comparison['ground_truth_models'])}")
        report.append(f"**Predicted Models:** {', '.join(comparison['predicted_models'])}")
        report.append(f"**Model Accuracy:** {comparison['model_accuracy']:.2%}")
        report.append(f"**Semantic Similarity:** {comparison['avg_semantic_similarity']:.2%}")
        
        report.append("\n**Question Comparisons:**")
        for j, q_comp in enumerate(comparison['question_comparisons']):
            report.append(f"- **Ground Truth {j+1}:** {q_comp['ground_truth']}")
            report.append(f"  **Predicted:** {q_comp['predicted']}")
            report.append(f"  **Similarity:** {q_comp['similarity']:.2%}")
            report.append(f"  **Match Type:** {q_comp['match_type']}\n")
    
    # Recommendations
    report.append("## Recommendations")
    if evaluation_results['model_classification_accuracy'] < 0.7:
        report.append("- The router struggles with accurate model classification. Consider retraining or improving the router's understanding of task categories.")
    if evaluation_results['avg_semantic_similarity'] < 0.8:
        report.append("- Question decomposition needs improvement. The router is breaking down questions in ways that semantically differ from the ground truth.")
    if evaluation_results['exact_match_rate'] < 0.3:
        report.append("- Very low exact matching suggests the router is paraphrasing or interpreting questions differently than intended.")
    
    return "\n".join(report)


    

In [34]:
# Evaluate router accuracy
evaluation_results = evaluate_router_accuracy(multi_hop_questions, results)

# Generate a report
report = generate_accuracy_report(evaluation_results)


print("Evaluation complete. Results saved to router_accuracy_results.json and router_accuracy_report.md")
print(f"Overall Model Classification Accuracy: {evaluation_results['model_classification_accuracy']:.2%}")
print(f"Average Semantic Similarity: {evaluation_results['avg_semantic_similarity']:.2%}")

Evaluation complete. Results saved to router_accuracy_results.json and router_accuracy_report.md
Overall Model Classification Accuracy: 87.00%
Average Semantic Similarity: 78.85%


In [35]:
with open("gpt_router_accuracy_results.json", "w") as f:
        json.dump(evaluation_results, f, indent=2)
    
with open("gpt_router_accuracy_report.md", "w") as f:
        f.write(report)

In [36]:
# mistral 7b as multi-hop router

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [40]:
def route_question_with_mistral(question, model, tokenizer):
    """
    Route the multi-hop question using Mistral 7B to determine which models to use
    and how to break down the question.
    """
    # Construct prompt for routing
    prompt = f"""<s>[INST] You are a financial question routing system.
    
I have a complex financial question that needs to be broken down into smaller questions.
Based on the question, determine:
1. Which two models should process this question in sequence
2. The first subquestion to ask
3. How to formulate the second subquestion using the answer from the first

Available models:
- FinQA: Good for basic financial calculations and accounting questions
- FinRed: Specialized in financial regulations and compliance
- FinForecaster: Best for market predictions and trend analysis

Question: {question}

Return your response in this exact JSON format:
{{
  "first_model": "MODEL_NAME",
  "first_subquestion": "SUBQUESTION_TEXT",
  "second_model": "MODEL_NAME", 
  "second_subquestion_template": "TEMPLATE_WITH_[ANSWER1]_PLACEHOLDER"
}}
[/INST]</s>"""

    # Generate response from Mistral
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the JSON part from the response
    try:
        # Find JSON content (simple approach, might need refinement)
        json_start = response.find("{")
        json_end = response.rfind("}") + 1
        if json_start != -1 and json_end != -1:
            json_str = response[json_start:json_end]
            routing_result = json.loads(json_str)
        else:
            # Fallback if JSON is not found
            routing_result = {
                "first_model": "FinQA",  # Default model as fallback
                "first_subquestion": question,
                "second_model": "FinForecaster", 
                "second_subquestion_template": "Based on [ANSWER1], what is the financial implication?"
            }
    except json.JSONDecodeError:
        # Fallback if JSON parsing fails
        routing_result = {
            "first_model": "FinQA",  # Default model as fallback
            "first_subquestion": question,
            "second_model": "FinForecaster", 
            "second_subquestion_template": "Based on [ANSWER1], what is the financial implication?"
        }
    
    return routing_result

def process_with_models(routing_result, models, tokenizers):
    """Process the question with the designated models in sequence"""
    first_model_name = routing_result["first_model"]
    second_model_name = routing_result["second_model"]
    
    # Process with first model
    first_subquestion = routing_result["first_subquestion"]
    first_model = models[first_model_name]
    first_tokenizer = tokenizers[first_model_name]
    
    # Generate answer with first model
    inputs = first_tokenizer(first_subquestion, return_tensors="pt").to(first_model.device)
    with torch.no_grad():
        outputs = first_model.generate(inputs["input_ids"], max_new_tokens=256)
    first_answer = first_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Format second question using the first answer
    second_subquestion = routing_result["second_subquestion_template"].replace("[ANSWER1]", first_answer)
    
    # Process with second model
    second_model = models[second_model_name]
    second_tokenizer = tokenizers[second_model_name]
    
    # Generate answer with second model
    inputs = second_tokenizer(second_subquestion, return_tensors="pt").to(second_model.device)
    with torch.no_grad():
        outputs = second_model.generate(inputs["input_ids"], max_new_tokens=256)
    second_answer = second_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return {
        "first_model": first_model_name,
        "first_subquestion": first_subquestion,
        "first_answer": first_answer,
        "second_model": second_model_name,
        "second_subquestion": second_subquestion,
        "second_answer": second_answer,
        "final_answer": second_answer  # May want to format this differently
    }

def process_multi_hop_questions(questions_dataset):
    models = {}
    tokenizers = {}
    results = []
    
    # Load Mistral router model once
    print("Loading Mistral 7B router model...")
    mistral_model, mistral_tokenizer = model, tokenizer
    print("Mistral router model loaded successfully")
    
    for idx, item in enumerate(questions_dataset):
        multi_hop_question = item['multi_hop_question']
        print(f"\nProcessing question {idx+1}: {multi_hop_question}")
        
        # Route the question using Mistral instead of previous router
        routing_result = route_question_with_mistral(multi_hop_question, mistral_model, mistral_tokenizer)
        
        # Add original question to routing result
        routing_result["original_question"] = multi_hop_question
        
        # Load models on demand
        for model_name in [routing_result["first_model"], routing_result["second_model"]]:
            if model_name not in models and model_name in model_loaders:
                print(f"Loading model: {model_name}")
                model, tokenizer = model_loaders[model_name]()
                models[model_name] = model
                tokenizers[model_name] = tokenizer
        
        # Process with models
        question_results = process_with_models(routing_result, models, tokenizers)
        
        # Combine routing result with processing results
        combined_results = {**routing_result, **question_results}
        
        # Compare to original subquestions if available
        if 'original_questions' in item:
            combined_results["original_subquestions"] = item['original_questions']
        
        results.append(combined_results)
        
        # Save results periodically
        if (idx + 1) % 5 == 0 or idx == len(questions_dataset) - 1:
            with open(f"multi_hop_results_{idx+1}.json", "w") as f:
                json.dump(results, f, indent=2)
    
    return results

In [47]:
mistral_results = []
for q in multi_hop_questions:
    r = route_question_with_mistral(q, model, tokenizer)
    mistral_results.append(r)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad

In [53]:
multi_hop_questions[0]

{'original_questions': [{'input': {'input': 'How can I improve my credit score quickly?',
    'output': 'It depends on your goals, but consider strategies like diversification, tax planning, and emergency savings.',
    'instruction': 'Utilize your financial knowledge, give your answer or opinion to the input question or subject. Answer format is not limited.',
    'module': 'FinQA'}},
  {'input': {'input': 'Will the Dow Jones rise or fall next week?',
    'output': '[Positive Developments]: Strong earnings, favorable macro trends. [Potential Concerns]: Technical resistance, volatility.\n\n[Prediction & Analysis]: Prediction: Up 2–4%. Analysis: Market sentiment and fundamentals suggest modest gains.',
    'instruction': 'Based on your knowledge, forecast the stock or crypto price movement and provide a brief analysis.',
    'module': 'Forecaster'}}],
 'multi_hop_question': 'How can I quickly improve my credit score in order to qualify for a better interest rate on a loan, and how might

In [60]:
mistral_results[0]

{'first_model': 'FinQA',
 'first_subquestion': {'original_questions': [{'input': {'input': 'How can I improve my credit score quickly?',
     'output': 'It depends on your goals, but consider strategies like diversification, tax planning, and emergency savings.',
     'instruction': 'Utilize your financial knowledge, give your answer or opinion to the input question or subject. Answer format is not limited.',
     'module': 'FinQA'}},
   {'input': {'input': 'Will the Dow Jones rise or fall next week?',
     'output': '[Positive Developments]: Strong earnings, favorable macro trends. [Potential Concerns]: Technical resistance, volatility.\n\n[Prediction & Analysis]: Prediction: Up 2–4%. Analysis: Market sentiment and fundamentals suggest modest gains.',
     'instruction': 'Based on your knowledge, forecast the stock or crypto price movement and provide a brief analysis.',
     'module': 'Forecaster'}}],
  'multi_hop_question': 'How can I quickly improve my credit score in order to qual

In [58]:
multi_hop_questions[0]['original_questions'][0]['input']['module']

'FinQA'

In [61]:
correct_first = 0
correct_second = 0
total = len(mistral_results)

for i in range(total):
    gt = multi_hop_questions[i]['original_questions']
    gt1 = gt[0]['input']['module']
    gt2 = gt[1]['input']['module']
    
    # Normalize ground-truth to match prediction format
    gt1 = gt1.replace('finRED', 'FinRed').replace('Forecaster', 'FinForecaster')
    gt2 = gt2.replace('finRED', 'FinRed').replace('Forecaster', 'FinForecaster')

    p1 = mistral_results[i]['first_model']
    p2 = mistral_results[i]['second_model']

    if p1 == gt1:
        correct_first += 1
    if p2 == gt2:
        correct_second += 1

# Compute accuracies
first_accuracy = correct_first / total
second_accuracy = correct_second / total
overall_accuracy = (correct_first + correct_second) / (2 * total)

print(f"\n✅ Router Evaluation:")
print(f"First model accuracy: {first_accuracy*100:.2f}%")
print(f"Second model accuracy: {second_accuracy*100:.2f}%")
print(f"Overall router accuracy: {overall_accuracy*100:.2f}%")



✅ Router Evaluation:
First model accuracy: 50.00%
Second model accuracy: 100.00%
Overall router accuracy: 75.00%


In [ ]:
def evaluate_multihop_response(predicted_answer, reference_answer, client, model="gpt-4o-mini"):
    """
    Evaluate a multi-hop answer using an LLM-based grading scheme via an OpenAI client.

    Args:
        predicted_answer (str): The system-generated multi-hop response.
        reference_answer (str): The concatenated ground-truth multi-hop response.
        client: An OpenAI API client instance.
        model (str): The model to use for evaluation (default: gpt-4o-mini).

    Returns:
        dict: Dictionary containing 'correctness' and 'completeness' scores (0-5 scale).
    """

    routing_prompt = f"""
You are an expert financial reasoning evaluator.

Below are two multi-hop answers:
- Predicted Answer: {predicted_answer}
- Reference Answer: {reference_answer}

Evaluate the Predicted Answer according to these two criteria:
1. Correctness (0-5): Logical and factual consistency compared to the Reference Answer.
2. Completeness (0-5): Thoroughness in covering all sub-tasks implied by the Reference Answer.

Respond ONLY with a JSON object in this format:
{{
    "correctness": <score between 0 and 5>,
    "completeness": <score between 0 and 5>
}}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful and precise grader."},
            {"role": "user", "content": routing_prompt}
        ],
        response_format={"type": "json_object"}  # Directly request structured JSON
    )

    scores = response.choices[0].message.content  # Already parsed as a dict due to response_format
    return scores


In [ ]:
correctness_scores = []
completeness_scores = []
total = len(mistral_results)

for i in range(total):
    predicted_answer = mistral_results[i]['predicted_answer']
    reference_answer = multi_hop_questions[i]['reference_answer']

    scores = evaluate_multihop_response(predicted_answer, reference_answer, client, model="gpt-4o-mini")
    correctness_scores.append(scores['correctness'])
    completeness_scores.append(scores['completeness'])

avg_correctness = sum(correctness_scores) / total
avg_completeness = sum(completeness_scores) / total
print(f"Average Correctness Score: {avg_correctness:.2f} / 5.0")
print(f"Average Completeness Score: {avg_completeness:.2f} / 5.0")